In [ ]:
!pip -q install transformers==4.17
!pip -q install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 32.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.3 MB/s eta 0:00:00


In [ ]:
from transformers import Trainer, TrainingArguments, RobertaTokenizer, RobertaModel, RobertaForSequenceClassification, AutoTokenizer
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [ ]:
splits = {'java_train': 'data/java_train-00000-of-00001.parquet', 'java_test': 'data/java_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["java_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["java_test"])

display(train_df.head())
display(test_df.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,0,Abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,Abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | Ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,AbstractContractGetFileStatusTest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,AbstractContractGetFileStatusTest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,AbstractContractGetFileStatusTest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,5,AbstractContractGetFileStatusTest.java,accept everything.,1,accept everything. | AbstractContractGetFileSt...,"[0, 0, 1, 0, 0, 0, 0]"
1,8,AbstractContractGetFileStatusTest.java,accept nothing.,1,accept nothing. | AbstractContractGetFileStatu...,"[0, 0, 1, 0, 0, 0, 0]"
2,12,AbstractContractGetFileStatusTest.java,equals the @code match field.,1,equals the @code match field. | AbstractContra...,"[0, 0, 1, 0, 0, 0, 0]"
3,17,AbstractS3ACommitterFactory.java,dynamically create the output committer based ...,1,dynamically create the output committer based ...,"[0, 0, 1, 0, 0, 0, 0]"
4,19,ApplicationConstants.java,environment for applications.,1,environment for applications. | ApplicationCon...,"[0, 0, 1, 0, 0, 0, 0]"


In [ ]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,0,abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,abstractcontractgetfilestatustest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,abstractcontractgetfilestatustest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,abstractcontractgetfilestatustest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,5,abstractcontractgetfilestatustest.java,accept everything.,1,accept everything | abstractcontractgetfilesta...,"[0, 0, 1, 0, 0, 0, 0]"
1,8,abstractcontractgetfilestatustest.java,accept nothing.,1,accept nothing | abstractcontractgetfilestatus...,"[0, 0, 1, 0, 0, 0, 0]"
2,12,abstractcontractgetfilestatustest.java,equals the @code match field.,1,equals the code match field | abstractcontract...,"[0, 0, 1, 0, 0, 0, 0]"
3,17,abstracts3acommitterfactory.java,dynamically create the output committer based ...,1,dynamically create the output committer based ...,"[0, 0, 1, 0, 0, 0, 0]"
4,19,applicationconstants.java,environment for applications.,1,environment for applications | applicationcons...,"[0, 0, 1, 0, 0, 0, 0]"


In [ ]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=7)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [ ]:
# class OneHotTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.get("labels")
#         outputs = model(**inputs)
#         logits = outputs.get("logits")

#         # Use BCEWithLogitsLoss for multi-label classification
#         loss_fct = nn.BCEWithLogitsLoss()
#         loss = loss_fct(logits, labels.float())

#         return (loss, outputs) if return_outputs else loss

class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Remove 'labels' from inputs before passing to model
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)  # Pass only the required inputs to the model
        logits = outputs.get("logits")

        if logits is None:
            # Handle the case where logits are missing, e.g., raise an exception or return a default loss
            raise ValueError("Logits are missing from the model output.")  # Or return a default loss

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [ ]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


# Optional compute_metrics function to get the accuracy, precision, recall, f1 for each of the individual labels
# without averaging them

# Function for individual metrics of each category
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# def compute_metrics(pred):
#     # Get predictions and apply sigmoid to get probabilities
#     logits = pred.predictions
#     preds = (logits > 0).astype(int)  # Threshold at 0 to get binary predictions

#     # Get true labels
#     labels = pred.label_ids

#     # Calculate precision, recall, and f1-score for each label without averaging
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
#     # Calculate accuracy per label (how many times each label is correctly predicted)
#     accuracy_per_label = (preds == labels).mean(axis=0)

#     # Prepare a dictionary to store metrics per category
#     metrics = {
#         'accuracy_per_label': accuracy_per_label,
#         'precision_per_label': precision,
#         'recall_per_label': recall,
#         'f1_per_label': f1
#     }

#     # Optionally, if you want to calculate an overall accuracy:
#     overall_accuracy = (preds == labels).all(axis=1).mean()  # This gives the fraction of samples where all labels are correctly predicted
#     metrics['overall_accuracy'] = overall_accuracy

#     return metrics

In [ ]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        warmup_steps=10,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,
        report_to="none",
    )

    trainer = OneHotTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best trial:")
print(study.best_trial)

[I 2024-12-10 13:03:53,609] A new study created in memory with name: no-name-bfe5a9e6-2aa6-470a-a5eb-4f3ea4a1656e
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=Tru

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.235695,0.931620,0.713992,0.696486,0.689972
1,No log,0.174753,0.954132,0.783040,0.797444,0.787664
2,0.268400,0.156800,0.956946,0.785234,0.813419,0.798608
3,0.268400,0.149074,0.957227,0.785719,0.815974,0.799808
4,0.268400,0.146390,0.957978,0.790373,0.815335,0.802060


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-12-10 13:10:10,728] Trial 0 finished with value: 0.8020595944716373 and parameters: {'learning_rate': 5.350684979513934e-06, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 0.0013815444704360312}. Best is trial 0 with value: 0.8020595944716373.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feat

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.147610,0.957602,0.777719,0.833227,0.802696
1,No log,0.098241,0.966513,0.901354,0.849201,0.867451
2,0.112600,0.101080,0.965106,0.898253,0.847284,0.868674
3,0.112600,0.094385,0.971110,0.900993,0.896486,0.897212
4,0.112600,0.096588,0.972610,0.906626,0.900958,0.902624
5,0.057400,0.092444,0.973267,0.909556,0.903514,0.906038


...

[I 2024-12-10 14:12:50,198] Trial 8 finished with value: 0.9085426739033046 and parameters: {'learning_rate': 3.751411178353134e-05, 'batch_size': 8, 'num_train_epochs': 5, 'weight_decay': 0.007424689981925305}. Best is trial 2 with value: 0.9107983091691144.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: Th

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,0.004500,0.139193,0.975049,0.912174,0.915016,0.913070
1,0.003900,0.140263,0.974768,0.910951,0.915016,0.912681
2,0.005400,0.144982,0.973830,0.907831,0.913099,0.910185
3,0.002700,0.144530,0.974299,0.909540,0.914377,0.911828
4,0.003100,0.143510,0.974017,0.907443,0.914377,0.910735


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-2500
Configuration sa

[I 2024-12-10 14:21:08,829] Trial 9 finished with value: 0.9107349958439056 and parameters: {'learning_rate': 1.3546589176360316e-06, 'batch_size': 4, 'num_train_epochs': 5, 'weight_decay': 0.032333901602290514}. Best is trial 2 with value: 0.9107983091691144.


Best trial:
FrozenTrial(number=2, state=TrialState.COMPLETE, values=[0.9107983091691144], datetime_start=datetime.datetime(2024, 12, 10, 13, 17, 39, 54064), datetime_complete=datetime.datetime(2024, 12, 10, 13, 24, 33, 249495), params={'learning_rate': 9.269199699140391e-06, 'batch_size': 8, 'num_train_epochs': 5, 'weight_decay': 0.0001396343462828426}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'num_train_epochs': IntDistribution(high=10, log=False, low=3, step=1), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=2, value=None)


In [12]:
best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=best_hyperparams["num_train_epochs"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    per_device_eval_batch_size=best_hyperparams["batch_size"] * 2,
    learning_rate=best_hyperparams["learning_rate"],
    weight_decay=best_hyperparams["weight_decay"],
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print("<<=================Training Time================>>")
print(f"Training time: {training_time:.2f} seconds")
print("<<==============================================>>")

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6091
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 1905


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.154818,0.975143,0.907634,0.920128,0.911586
2,0.003700,0.152314,0.974393,0.911786,0.914377,0.912007
3,0.003500,0.155530,0.972704,0.898135,0.917572,0.907535
4,0.001900,0.153187,0.973642,0.902353,0.916933,0.908985


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 16
Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 16
Saving model checkpoint to ./best_results/checkpoint-1000
Configuration saved in ./best_results/checkpoint-1000/config.json
Model weights saved in ./best_results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 16
Saving model checkpoint to ./best_results/checkpoint-1500
Configuration saved in ./best_results/checkpoint-1500/config.json
Model weights saved in ./best_results/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 16


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.154818,0.975143,0.907634,0.920128,0.911586
2,0.003700,0.152314,0.974393,0.911786,0.914377,0.912007
3,0.003500,0.155530,0.972704,0.898135,0.917572,0.907535
4,0.001900,0.153187,0.973642,0.902353,0.916933,0.908985
5,0.001900,0.152871,0.974205,0.907008,0.916294,0.911419


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




<<=================Training Time================>>
Training time: 405.91 seconds
<<==============================================>>


In [13]:
eval_results = trainer.evaluate()
print(eval_results)

***** Running Evaluation *****
  Num examples = 1523
  Batch size = 16


{'eval_loss': 0.15287107229232788, 'eval_accuracy': 0.9742050464309164, 'eval_precision': 0.9070082034907487, 'eval_recall': 0.91629392971246, 'eval_f1': 0.91141935663054, 'eval_runtime': 5.8242, 'eval_samples_per_second': 261.495, 'eval_steps_per_second': 16.483, 'epoch': 5.0}


In [14]:
!mkdir 'best_model_roberta-large_java'
!mkdir 'best_model_roberta-large_java_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_roberta-large_java')
tokenizer.save_pretrained('./best_model_roberta-large_java_tokenizer')

# Load model and tokenizer
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model = RobertaForSequenceClassification.from_pretrained('./best_model_roberta-large_java')
tokenizer = RobertaTokenizer.from_pretrained('./best_model_roberta-large_java_tokenizer')

Saving model checkpoint to ./best_model_roberta-large_java
Configuration saved in ./best_model_roberta-large_java/config.json
Model weights saved in ./best_model_roberta-large_java/pytorch_model.bin
tokenizer config file saved in ./best_model_roberta-large_java_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_roberta-large_java_tokenizer/special_tokens_map.json
loading configuration file ./best_model_roberta-large_java/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "lab

## Model Testing

In [20]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [21]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

In [22]:
!zip -r best_model_codebert-base_java.zip './best_model_roberta-large_java'
!zip -r best_model_codebert-base_java_tokenizer.zip './best_model_roberta-large_java_tokenizer'

  adding: best_model_roberta-large_java/ (stored 0%)
  adding: best_model_roberta-large_java/training_args.bin (deflated 51%)
  adding: best_model_roberta-large_java/pytorch_model.bin (deflated 14%)
  adding: best_model_roberta-large_java/config.json (deflated 55%)
  adding: best_model_roberta-large_java_tokenizer/ (stored 0%)
  adding: best_model_roberta-large_java_tokenizer/tokenizer_config.json (deflated 80%)
  adding: best_model_roberta-large_java_tokenizer/vocab.json (deflated 63%)
  adding: best_model_roberta-large_java_tokenizer/special_tokens_map.json (deflated 84%)
  adding: best_model_roberta-large_java_tokenizer/merges.txt (deflated 53%)


In [23]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

# Measure inference time
start_time = time.time()

# Predict on the test dataset
predictions = trainer.predict(test_dataset)

# Calculate elapsed time
end_time = time.time()
inference_time = end_time - start_time

# Compute metrics using the `compute_metrics` function
metrics = compute_metrics(predictions)

print("<<==============================================>>")
# Display metrics and inference time
print("<<=================Evaluation Metrics================>>")
print("Evaluation Metrics:", metrics)
print("<<==============================================>>")
print("<<=================Inference Time================>>")
print(f"Inference Time: {inference_time:.2f} seconds")
print("<<==============================================>>")

***** Running Prediction *****
  Num examples = 1725
  Batch size = 16


<<==============================================>>
<<=================Evaluation Metrics================>>
Evaluation Metrics: {'accuracy': 0.9429399585921325, 'precision': 0.8156749764817066, 'recall': 0.8071387449625792, 'f1': 0.8089909457616646}
<<==============================================>>
<<=================Inference Time================>>
Inference Time: 6.83 seconds
<<==============================================>>


In [24]:
# Transfer the model to Google drive

from google.colab import drive
drive.mount('/content/drive')

!cp best_model_roberta-large_java.zip "/content/drive/MyDrive/FYP/Revised/Models/roberta-large_java/"
!cp best_model_roberta-large_java_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/roberta-large_java_Tokenizer/"

Mounted at /content/drive


# **Load and Test Model**

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

# Replace 'path/to/checkpoint-folder' with the actual path to your checkpoint folder.
model_folder = '/content/drive/MyDrive/FYP/Revised/Models/roberta-large_java/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/roberta-large_java_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/roberta-large_java/best_model_roberta-large_java.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/roberta-large_java_Tokenizer/best_model_roberta-large_java_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/FYP/Revised/Models/roberta-large_java/best_model_roberta-large_java.zip
replace ./best_model_roberta-large_java/training_args.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_roberta-large_java/training_args.bin  
  inflating: ./best_model_roberta-large_java/pytorch_model.bin  
  inflating: ./best_model_roberta-large_java/config.json  
Archive:  /content/drive/MyDrive/FYP/Revised/Models/roberta-large_java_Tokenizer/best_model_roberta-large_java_tokenizer.zip
replace ./best_model_roberta-large_java_tokenizer/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_roberta-large_java_tokenizer/tokenizer_config.json  
  inflating: ./best_model_roberta-large_java_tokenizer/vocab.json  
  inflating: ./best_model_roberta-large_java_tokenizer/special_tokens_map.json  
  inflating: ./best_model_roberta-large_java_tokenizer/merges.txt  


In [27]:
best_model_roberta_large_java = 'best_model_roberta-large_java'
best_model_roberta_large_java_tokenizer = 'best_model_roberta-large_java_tokenizer'

In [28]:
# Load the model and tokenizer from the checkpoint
model = RobertaForSequenceClassification.from_pretrained(best_model_roberta_large_java)
tokenizer = RobertaTokenizer.from_pretrained(best_model_roberta_large_java_tokenizer)

text = "for detecting automatically generated fields."
start_time = time.time()
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
end_time = time.time()
inference_time = end_time - start_time
print("<<=================Inference Time================>>")
print(f"Inference Time: {inference_time:.2f} seconds")
print("<<==============================================>>")
output

loading configuration file best_model_roberta-large_java/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_v

<<=================Inference Time================>>
Inference Time: 0.16 seconds
<<==============================================>>


SequenceClassifierOutput(loss=None, logits=tensor([[ 4.1450, -9.7462,  2.2532, -6.4078, -7.5083, -9.1871, -7.2240]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [29]:
logits = output.logits

# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)

# Define a threshold to determine if a class is positive
threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

# Convert indices to a list for output
predicted_classes = (predicted_indices + 1).tolist()

# Format the output to display classes
if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 1, 3
